<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="./assets/holoviz-logo-unstacked.svg" />


# Seeing the needle AND the haystack: single-datapoint selection for billion-point datasets

*How to view millions or billions of datapoints while allowing precise selection and exploration of the details of individual datapoints.*

<div style="margin: 10px">
    <img style="margin:8px; display:inline; object-fit:scale-down; max-height:65px" src="./assets/datashader.png"/>
    <img style="margin:8px; display:inline; object-fit:scale-down; max-height:65px" src="./assets/bokeh.png"/>
    <img style="margin:8px; display:inline; object-fit:scale-down; max-height:65px" src="./assets/holoviews.png"/>
    <img style="margin:8px; display:inline; object-fit:scale-down; max-height:70px" src="./assets/numba.png"/>
</div>


## 1: Visualizing large data is challenging

#### Large datasets containing millions or billions of records have become common but there are very few tools that can visualize such datasets in their entirety. Even fewer tools also enable interactive exploration and inspection of entire datasets.

* Data is cheap and plentiful but insight is expensive.
* Most plotting tools are not designed to visualize gigabytes of data at a time.
* *Overplotting* is one problem that arises as data density increases.
* Expressing *large dynamic ranges* in the data can also be a problem as sample densities increase.
* WebGL approaches (e.g. [`jupyter_scatter`](https://github.com/flekschas/jupyter-scatter)) improve the situation but are limited to the memory and compute resources of a browser tab

### Datashader (datashader.org)
 
In 2016, the datashader project set out to see if visualizing billions of samples at once in a meaningful way was possible. Using numba (numba.org) for speed, datashader demonstrated that meaningful, beautiful visualizations were indeed feasible. Here are some examples:

<img src="https://datashader.org/assets/images/usa_census.jpg" width=50%></img>
<img src="https://datashader.org/assets/images/nyc_races.jpg" width=50%></img>

While the examples in this talk are geographic in nature, datashader can be used for anything! Here are some pretty mathematical attractors:

<img src="https://datashader.org/assets/images/sym_attractors.jpg" width=50%></img>


Datashader solves the two key problems with large data as follows:

* Overplotting: samples are *rasterized* to an image on screen by representing the data as a 2D histogram.
* Large dynamic range (parameter-free): colormapping using *histogram equalization* makes much better use of colormaps without having any parameters to tune.

While these are certainly pretty images in which you can spot patterns, it is not possible to query the data in any meaningful way (as these are static images).

## 2. Example dataset: herring gull tracking

To illustrate the datashader API, we will now load a relatively large public dataset. In particular, the [Research Institute for Nature and Forest (INBO)](https://www.vlaanderen.be/inbo/en-gb/homepage/) has been monitoring GPS locations of herring gulls breeding at the North Sea coast of Belgium since 2013. We will now load the 2018 portion of this data:

In [ ]:
import pandas as pd
import datashader as ds
import colorcet

In [ ]:
df = pd.read_parquet('./data/gulls.parq') # Small for datashader, big for other approaches!
print(f'There are {len(df)/1e6:.2f} million samples in this dataset')
print(f'The columns of this dataframe are: {", ".join(df.columns)}')

Here is an example of some of the samples in this dataset:

In [ ]:
df.head()

In [ ]:
cvs = ds.Canvas(plot_width=850, plot_height=500)
agg = cvs.points(df, 'longitude', 'latitude')
img = ds.tf.shade(agg, cmap=colorcet.CET_L4, how='eq_hist')
ds.tf.set_background(img, "black") 

This output is simple an RGB image which illustrates the following features of datashader:

*Datashader features (2016)*

| Whole dataset | No overplotting | Full dynamic range |
| --- | --- | --- |
| ✔️ | ✔️ | ✔️ |

However, it is equally clear to anyone used to client-side, interactive plotting that a lot of key features required for data exploration are missing.

## 3. Comparison to interactive plotting

As the visualization above is clearly very limited in its interactivity, it is instructive to compare this output with what is possible with a interactive plotting library. Here we will use Bokeh (bokeh.org) to plot a small subset of the data.

In [ ]:
subset = df.sample(frac=0.005)
print('There are {:d}  samples in this subset of the dataset '.format(len(subset)))

In [ ]:
import hvplot.pandas
subset.hvplot(x='easting', y='northing', tiles='ESRI', kind='points',
              xlabel='longitude', ylabel='latitude', width=900, height=400, hover_cols=list(subset.columns))

Tthe overplotting issue are apparent even with this small slice of the data and we are unable to view the full data distribution. However, we have gained many useful interactive features for exploring the data samples:

*Bokeh features (2016)*

| Whole dataset | No overplotting | Full dynamic range | Panning | Zooming | Tiles | Query | Inspect |
| --- | --- | --- | --- | --- | --- | --- | --- |
| ❌ | ❌ | ❌ | ✔️| ✔️| ✔️| ❌| ✔️|

* **'Querying'** means using all the samples found to build a summary.
* **'Inspect'** means being able to see a subset of representative samples.


## 4. Interactive zooming and panning

In 2017, HoloViews added interactive support for datashader using its streams system which integrates with the Bokeh event system. In 2018, the hvplot (hvplot.holoviz.org) project exposed this functionality with a simpler API that is more familiar to many users.

### Using datashade (RGB image)

With hvplot, you can apply datashader with server-side `eq_hist` (histogram equalization) colormapping with the option `datashade=True`:

In [ ]:
df.hvplot(x='easting', y='northing', tiles='ESRI', kind='points', 
              xlabel='longitude', ylabel='latitude', datashade=True, 
          width=900, height=400, cmap=colorcet.CET_L4 )

Many of the most critical interactive features are now available but there is still no colorbar and no point querying or inspection. This is because datashader is generating an RGB image which is an inappropriate format for these features:

*HoloViews (2017), hvPlot (2018)*

| Whole dataset | No overplotting | Full dynamic range | Panning | Zooming | Tiles | Colorbar | Query | Inspect |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| ✔️ | ✔️ | ✔️ | ✔️| ✔️| ✔️| ❌ | ❌ | ❌ |


### Using rasterize

Alternatively, you can request that datashader skip the server-side colormapping to output the raw aggregated values to the browser, allowing client-side colormapping. This is achieved with the option `rasterize=True`:

In [ ]:
df.hvplot(x='easting', y='northing', tiles='ESRI', kind='points', 
          xlabel='longitude', ylabel='latitude', rasterize=True, 
          cmap=colorcet.CET_L4 , colorbar=True, cnorm='log', width=900, height=400) 
# Until recently, could only use linear or log colormapping in cnorm

Now we have client-side colormapping, the colorbar can be enabled again. The issue now is that until 2021, only linear and log colormapping was available which do not expose the full dynamic range of the data. In addition, the only inspection available is of the aggregated datashader output and not the original data associated with each sample.

*HoloViews (2017), hvPlot (2018)*


| Whole dataset | No overplotting | Full dynamic range | Panning | Zooming | Tiles | Colorbar | Query | Inspect |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| ✔️ | ✔️ | ❌ | ✔️| ✔️| ✔️| ✔️ | ❌ | ❌ |


### Bokeh `eq_hist` colormapping

In Bokeh 2.2, support for histogram equalization colormapping (i.e. `eq_hist`) was added. This means that this colormapping approach can now be used on the output of `rasterize`, exposing the full dynamic range of the data together with a colorbar:

In [ ]:
df.hvplot(x='easting', y='northing', tiles='ESRI', kind='points', 
          xlabel='longitude', ylabel='latitude', rasterize=True, 
          cmap=colorcet.CET_L4, colorbar=True, cnorm='eq_hist', width=900, height=400) 

Now we have all the desired interactive plotting features familiar with small datasets except for the ability to inspect individual samples:

*HoloViews (2021), hvPlot (2021)*

| Whole dataset | No overplotting | Full dynamic range | Panning | Zooming | Tiles | Colorbar | Query | Inspect |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| ✔️ | ✔️ | ✔️ | ✔️| ✔️| ✔️| ✔️ | ❌ | ❌ |


As this table shows, every feature excluding inspection can be achieved with the current [hvplot API](https://hvplot.holoviz.org/). You can learn more about what is possible with `hvplot` at our tutorial tomorrow afternoon!

## 5. Querying data samples

**Problem statement**: *When I see something interesting in the rasterized display, how can I easily view information based **all** the datapoints in the area of interest?*

While all datapoints in the region of interest are available, the default behavior is to display information about the **closest** sample to the cursor given its position on a raster image. 

With HoloViews, you can follow the pipeline backwards from the rasterized output of datashader back to the original raw data. This means you can first get the mouse position in the data coordinates of the plot. Then by following the pipeline back to the original dataframe, you can then filter the data down to a small window around that cursor location to find a small number of candidate samples and then the single closest sample. This sample can then be displayed as a Bokeh glyph with the appropriate hover information.

Using the HoloViews API, we can declare a tile source and then wrap our raw datas in a `hv.Points` object which we then rasterize and overlay over this tile source:


In [ ]:
import holoviews as hv
esri = hv.element.tiles.ESRI().redim(x='easting', y='northing').opts(xlim=(1.5e4,4.22e5), ylim=(6.25e6,6.7e6))
points = hv.Points(df, kdims=['easting', 'northing'])
raster = hv.operation.datashader.rasterize(points).opts(cmap=colorcet.CET_L4, 
                                                        cnorm='eq_hist', width=800, height=400, colorbar=True)
esri * raster

Using the `inspect` operation we can build a `highlight` object that makes inspection information available:

In [ ]:
highlight   = hv.operation.datashader.inspect(raster)
esri * raster * highlight.opts(marker='o', size=10, fill_alpha=0, color='white',  tools=["hover"])

Inspections also works for polygon/shape data as demonstrated in the [`nyc_buildings`](https://examples.pyviz.org/nyc_buildings/nyc_buildings.html#nyc-buildings-gallery-nyc-buildings) example (interactive exploration of buildings in NYC)

### Pros and cons of spatial querying

This querying approach does not rely on any mechanisms in datashader. As such is has the following pros and cons:

*Pros*:

* Access to *all* samples in a defined area around the cursor
* This allows you to compute accurate aggregates, no matter how many samples contributed to a pixel.

Cons:

* Has to run a slow spatial filter on the whole dataframe to find relevant samples (spatial indexing can help)
* Works interactively at WebGL scales but does not scale to the really massive data sizes datashader can handle
* As a result, this cannot be enabled by default

With this feature  we now have many of the interactive plotting conveniences familiar when working with small data but for much larger datasets:

*HoloViews (2021, version 1.14.4 with further integration and improvements planned)*

| Whole dataset | No overplotting | Full dynamic range | Panning | Zooming | Tiles | Colorbar | Query | Inspect |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| ✔️ | ✔️ | ✔️ | ✔️| ✔️| ✔️| ✔️ | ✔️ | ❌ |

Due to this scaling issue, we cannot use this approach for hover inspection when there are >10 million samples.

## 6. Instant hover inspection

*Datashader (2023)*

*(Will be available in an upcoming `holoviews` and `hvplot` release soon!)*

**Problem statement**: *When I see something interesting in the rasterized display, how can I easily see data about a representative/informative sample for any data set size?

Using Datashader support, the slow step of spatial filtering can be skipped by computing the appropriate indices in the same pass that builds the visual image.


## How it works - index arrays

Datashader supports 'selectors' which are used to find the indexes of the inputs, which is then used to lookup the other values in the data. Effort is made to ensure all aggregation requires only a single datashader pass (i.e. the data is traversed once).

### Selectors

Here is the current set of selectors in datashader that work with this approach:

- `ds.min(<column>)`: Find the index of the sample with the minimum `<column>` value
- `ds.max(<column>)`: Find the index of the sample with the maximum `<column>` value
- `ds.first(<column>)`: Find the index of the first sample aggregated from `<column>`
- `ds.last(<column>)`: Find the index of the last sample aggregated from `<column>`

Using the `ds.max` selector, here is the index array for the gulls with the maximum altitude above sea-level:

In [ ]:
raster = hv.operation.datashader.rasterize(hv.Points(df, ['easting', 'northing']), 
                                           selector=ds.max("height-above-msl"), dynamic=False)
hv.Image(raster.data['index']).opts(width=800, height=400, tools=['hover'])

This index array allows us to expose hover information without a separate (slow) spatial filtering step. Here is the gull visualization where the hover information shows the seagull with the highest altitude above mean sea level:

In [ ]:
points = hv.Points(df, ['easting', 'northing'])
raster = hv.operation.datashader.rasterize(points, selector=ds.max("height-above-msl")).opts(
    tools=["hover"], colorbar=True, cmap=colorcet.CET_L4,  width=800, height=400, cnorm="eq_hist")
esri * raster



| Whole dataset | No overplotting | Full dynamic range | Panning | Zooming | Tiles | Colorbar | Query | Inspect |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| ✔️ | ✔️ | ✔️ | ✔️| ✔️| ✔️| ✔️ | ✔️ | ✔️ |


### Non-geo example of dimensionality reduction

Demo!

### Example with 211 million samples...

* [`ship_traffic`](https://examples.pyviz.org/ship_traffic/ship_traffic.html#ship-traffic-gallery-ship-traffic): This example shows the inspection approach working on 211 million AIS pings for ships sailing around the west coast of the US. This are transponder pings that record (among other things) the ship's position, type and heading. This example uses custom hooks to show a table of the inspected point as well as a photograph of the ship (when available).


## 7. Using panel to turn this into a servable dashboard

This new functionality is part of the wider HoloViz (holoviz.org) ecosystem which allows you to turn these kind of visualizations into dashboards using panel (panel.holoviz.org).

The `panel serve` command will serve any panel marked `.servable()` in the notebook as a dashboard. To learn more about [panel](https://panel.holoviz.org/) and how to quickly turn notebooks into dashboards, visit the [holoviz tutorial](https://holoviz.org/tutorial/index.html).

In [ ]:
import panel as pn
template = pn.template.FastListTemplate(title="Gull tracking dashboard")

points = hv.Points(df, kdims=['easting', 'northing'])
raster = hv.operation.datashader.rasterize(points, selector=ds.max("height-above-msl")).opts(
    tools=["hover"], colorbar=True, cmap=colorcet.CET_L4,  responsive=True, cnorm="eq_hist")
esri * raster
overlay = esri * raster.opts(cmap='fire', cnorm='eq_hist', min_height=400)
description = """
This dashboard shows how easy it is to go from an aggregate view of 2.4 million points
down to the details of a single GPS sample.
"""
template.main.append(pn.Column(description, overlay, sizing_mode='stretch_both' ))
template.servable();

## 8. Ongoing work and Conclusion

Since the initial release of datashader in 2016, the HoloViz ecosystem has worked to make interactive plotting features available for working with large datasets. With the addition of client-side `eq_hist` colormapping in Bokeh, the new `inspect` operation and instant hover, it is now possible to work with large data in the same way as you can currently work with smaller datasets.

This means you can see *all* your data (gigabytes at a time) while also allowing you to interactively link and drilldown to view the detailed information for subsets and individual data points.


Of course there are always more improvements in progress that will be available soon:

* Bokeh now supports categorical color mixing! This feature will be available in `holoviews` and `hvplot` soon:

<img width='70%' src="./assets/colormixing.png"/>

* Datashader now has antialised lines and aims for first-class support for timeseries data. Work is ongoing to allow antialiasing as well as the `where` aggregator.


### To learn more about HoloViz, please visit `holoviz.org` and all are welcome to join the [EuroPython HoloViz Panel Sprints!](https://ep2023.europython.eu/sprints#holoviz-panel)

## 9. Acknowledgements

HoloViz is now a [NUMFOCUS sponsored project!](https://numfocus.org/project/holoviz)

<img src="./assets/numfocus.png"/>

<img src="./assets/contributors.svg"/>

# 10. Questions?


## Preprocessing the data

The raw data can be found online [here](https://zenodo.org/record/3541812/files/HG_OOSTENDE-gps-2018.csv). If you download this CSV file to `./data/HG_OOSTENDE-gps-2018.csv`, you can apply the basic cleanup needed to generate `gulls.parq` using the script included here:

<br>

<details>
    <br><tt>
import numpy as np <br>
import pandas as pd <br>
from holoviews.util.transform import lon_lat_to_easting_northing <br><br>

usecols = ['location-long', 'location-lat', 'external-temperature', 'heading', 'height-above-msl']<br>
df = pd.read_csv('./data/HG_OOSTENDE-gps-2018.csv', usecols=usecols)<br>
df.columns = ['longitude', 'latitude', 'temperature', 'heading', 'height-above-msl']<br><br>

sliced = df[(df['longitude'] > 0) & (df['longitude'] < 4) & (df['latitude'] > 49) & (df['latitude'] < 51.5)]<br><br>

x,y = lon_lat_to_easting_northing(sliced.longitude, sliced.latitude)<br>
projected = sliced.join([pd.DataFrame({'easting': x}), pd.DataFrame({'northing': y})])<br>
projected.to_parquet('./data/gulls.parq', 'fastparquet', compression='gzip',<br>
                     file_scheme='simple')<br>
</tt>
</details>